In [13]:
import os
os.chdir(os.path.join(os.getcwd(), '..','..', '..'))
import weka.core.jvm as jvm
import numpy as np
import csv
import pandas as pd
from weka.classifiers import Classifier
from weka.core.converters import Loader
from weka.core.converters import Loader
from weka.classifiers import Evaluation
from weka.core.classes import Random

from dataset.preprocessing.tf_idf_all_feature_matrix_gen import TFIDFFeatureGeneration


jvm.start()
loader = Loader(classname="weka.core.converters.ArffLoader")


INFO:weka.core.jvm:JVM already running, call jvm.stop() first


In [27]:
class J48:
    def __init__(self, train_data):
        self.j48 = Classifier(classname="weka.classifiers.trees.J48")
        self.j48.build_classifier(train_data)
        self.evaluator = Evaluation(train_data)

        
        # print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)
    
    def test_and_evaluate(self, train_data):
        self.evaluator.crossvalidate_model(self.j48, train_data, 10, Random(1))
        f1_macro = self.evaluator.weighted_f_measure
        f1_micro = self.evaluator.f_measure(1)
        # print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro

In [28]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']

column_headings = ["Morbidity Class", "DT_Macro F1", "DT_Micro F1"]

with open("./results/tf-idf-features/performance_J48.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    

In [30]:
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    f1_macro_list = []
    f1_micro_list = []
    train_data = loader.load_file(f"./dataset/train/train_{morbidity}_tfidf.arff")
    train_data.class_is_last()
    try:
        j48_obj = J48(train_data)
        f1_macro, f1_micro = j48_obj.test_and_evaluate(train_data)
    except Exception as ex:
        print(str(ex))
        f1_macro = 1
        f1_micro = 1
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf-features/performance_J48.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/tf-idf-features/performance_J48.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
Macro F1 score: 0.967920420055783 and Micro F1 Score 0.8656716417910447
CAD
Macro F1 score: 0.854211455359452 and Micro F1 Score 0.8761609907120742
CHF
weka.classifiers.trees.J48: Cannot handle unary class!
Macro F1 score: 1 and Micro F1 Score 1
Depression


Exception in thread "Thread-1" weka.core.UnsupportedAttributeTypeException: weka.classifiers.trees.J48: Cannot handle unary class!
	at weka.core.Capabilities.test(Capabilities.java:1045)
	at weka.core.Capabilities.test(Capabilities.java:1256)
	at weka.core.Capabilities.test(Capabilities.java:1138)
	at weka.core.Capabilities.testWithFail(Capabilities.java:1468)
	at weka.classifiers.trees.J48.buildClassifier(J48.java:277)


Macro F1 score: 0.8486251768781816 and Micro F1 Score 0.6454183266932271
Diabetes
Macro F1 score: 0.9118165784832452 and Micro F1 Score 0.9368686868686869
Gallstones
Macro F1 score: 0.7691496543535847 and Micro F1 Score 0.2391304347826087
GERD
Macro F1 score: 0.6495345177166398 and Micro F1 Score 0.2868852459016393
Gout
Macro F1 score: 0.785234899328859 and Micro F1 Score 0.1794871794871795
Hypercholesterolemia
Macro F1 score: 0.8048738734088037 and Micro F1 Score 0.8085937499999999
Hypertension
Macro F1 score: 0.8861496713329197 and Micro F1 Score 0.9302325581395349
Hypertriglyceridemia
Macro F1 score: 0.9072232298239227 and Micro F1 Score 0.13333333333333333
OA
Macro F1 score: 0.7451327433628319 and Micro F1 Score 0.2653061224489796
Obesity
Macro F1 score: 0.9548031823313856 and Micro F1 Score 0.9478079331941545
OSA
Macro F1 score: 0.9016949152542373 and Micro F1 Score 0.6547619047619048
PVD
Macro F1 score: 0.9053311615187141 and Micro F1 Score 0.7005649717514124
Venous-Insufficiency

In [8]:
jvm.stop()

0,0.014637,0.015228,0.013877,0,0,0,0,0.011786,0,0,0,0.038485,0,0,0,0.007837,0.017422,0.062747,0.027826,0.015317,0.011053,0,0.007327,0.009066,0,0,0.046689,0.333541,0.048852,0,0,0,0,0,0.01655,0,0.010052,0,0.094985,0,0.010375,0.048233,0,0,0,0,0.026292,0,0,0.059873,0.013733,0,0,0,0,0,0.339955,0,0,0,0,0,0.025705,0.074316,0,0.052802,0,0.066151,0,0,0,0,0,0,0.053292,0.064142,0,0,0,0,0,0,0.016442,0,0,0.027541,0.0222,0,0,0,0,0,0.030281,0,0.019714,0,0.00805,0,0,0,0.061767,0,0.010396,0.01588,0,0,0.023833,0,0,0.016885,0.010375,0,0,0.045291,0.015978,0.014597,0,0.009272,0,0,0,0.027754,0,0.013192,0,0.022625,0,0,0,0,0.129645,0.012828,0,0.02156,0.012672,0.060831,0.013166,0,0,0,0,0.040586,0.011193,0.014862,0,0.006459,0.0144,0,0,0.006414,0.038485,0.01618,0.070273,0.00853,0.03377,0.014678,0,0.016496,0,0,0,0.014208,0.009934,0.014759,0.093378,0,0.018147,0,0,0.013806,0.023676,0,0,0,0,0,0.009395,0,0.016885,0.042177,0.010355,0.014718,0.029274,0,0,0,0,0,0,0.014801,0,0,0.015929,0.013328,0.024046,0.021477,0,0.0419